In [1]:
import numpy as np
import pandas as pd
import os
from evaluation import *
import random

In [2]:
result='/mnt/user/drive/My Libraries/tutorials&explorations/V1criticality/data/2022-06-26_c_ee11.57199543729056_c_ei10.307919639315172_dt1_results.csv'
df = pd.read_csv(result, header = None)
data = df[1].to_numpy()
time = df[0].to_numpy()

In [3]:
rand_sig = np.random.uniform(-1, 1, time.shape[0])
rand_sig.shape

(1000000,)

In [4]:
df0 = pd.DataFrame({'time':time, 'data':rand_sig})
result_name = '/mnt/user/drive/My Libraries/tutorials&explorations/V1criticality/data/2022-07_15_randm4_results.csv'
df0.to_csv(result_name, mode='a', index=False, header=False)

In [5]:
dfa_analysis(result_name, dt=1, not_remove = True, save_inter=True)

Creating RawArray with float64 data, n_channels=1, n_times=1000000
    Range : 0 ... 999999 =      0.000 ...   999.999 secs
Ready.
Creating RawArray with float64 data, n_channels=1, n_times=1000000
    Range : 0 ... 999999 =      0.000 ...   999.999 secs
Ready.
Creating RawArray with float64 data, n_channels=1, n_times=1000000
    Range : 0 ... 999999 =      0.000 ...   999.999 secs
Ready.
Creating RawArray with float64 data, n_channels=1, n_times=1000000
    Range : 0 ... 999999 =      0.000 ...   999.999 secs
Ready.
Creating RawArray with float64 data, n_channels=1, n_times=1000000
    Range : 0 ... 999999 =      0.000 ...   999.999 secs
Ready.
Creating RawArray with float64 data, n_channels=1, n_times=1000000
    Range : 0 ... 999999 =      0.000 ...   999.999 secs
Ready.


(0.4081313433146249,
 {'delta': 0.5400541215762146,
  'theta': 0.5449977626190278,
  'alpha': 0.49366199567089325,
  'beta': 0.5276907158548721,
  'gamma': 0.4796421657082125,
  'raw': 0.5078950076843392,
  'dfa_this_trial': 0.5078950076843392},
 58.10546875)

### Get the psd of old data

In [7]:
result_name = '/mnt/user/drive/My Libraries/tutorials&explorations/V1criticality/data/2022-06-27_c_ee6_c_ei22_results.csv'
df = pd.read_csv(result_name, header = None)
data = df[1].to_numpy()
peak = get_psd_peak(data, dt=1,save_path=result_name)

### Get the filtered as well as hilbert transformed signal of old data

2022-06-27_c_ee6_c_ei22_results.csv
2022-07_15_randm2_results.csv
2022-06-26_c_ee11.57199543729056_c_ei10.307919639315172_dt1_results.csv

In [22]:
result_name = '/mnt/user/drive/My Libraries/tutorials&explorations/V1criticality/data/2022-06-26_c_ee11.57199543729056_c_ei10.307919639315172_dt1_results.csv'
dt=1
df = pd.read_csv(result_name, header = None)
data = df[1].to_numpy()
raw = dfa.load_data([data], sfreq = 1000/dt)

raw.filter(0.5, 4, fir_window='hamming', fir_design="firwin", verbose=0)
filtered_d, filtered_t = raw[:]
filtered_sig = filtered_d[0] - np.mean(filtered_d[0])
df_f = pd.DataFrame({'time':filtered_t, 'data':filtered_sig})
filtered_name=result_name[:result_name.find('results.csv')]+'filtered'+str(0.5)+'_'+str(4)+'.csv'
df_f.to_csv(filtered_name, mode='w', index=False, header=False)

filtered = dfa.load_data([filtered_sig], sfreq = 1000/dt)
filtered.apply_hilbert(envelope=True)
hilb, filtered_t = raw[:]
df_h = pd.DataFrame({'time':filtered_t, 'data':hilb[0]})
hilbert_name=result_name[:result_name.find('results.csv')]+'hilbert'+str(0.5)+'_'+str(4)+'.csv'
df_h.to_csv(hilbert_name, mode='w', index=False, header=False)

Creating RawArray with float64 data, n_channels=1, n_times=1000000
    Range : 0 ... 999999 =      0.000 ...   999.999 secs
Ready.
Creating RawArray with float64 data, n_channels=1, n_times=1000000
    Range : 0 ... 999999 =      0.000 ...   999.999 secs
Ready.


### Get the fitting model of old data

In [58]:
# %matplotlib inline

In [28]:
result_name = '/mnt/user/drive/My Libraries/tutorials&explorations/V1criticality/data/2022-06-26_c_ee11.57199543729056_c_ei10.307919639315172_dt1_results.csv'
dt=1
df = pd.read_csv(result_name, header = None)
data = df[1].to_numpy()
raw = dfa.load_data([data])
dfa_exp, _, x, y, model, residual = dfa.compute_DFA(raw, return_fitting=True, l_freq=0.5, h_freq=4)
# plt.figure()#figsize=(9.4,5))
# plt.plot(x,y)
# plt.xlabel('lg(window size/sec)')
# plt.ylabel('lg(flucturation)')
# poly1d_fn = np.poly1d(model) 
# plt.plot(x,y, 'ro', x, poly1d_fn(x))
# plt.show()
# plt.savefig('/mnt/user/drive/My Libraries/tutorials&explorations/V1criticality/data/2022-06-27_c_ee6_c_ei22_results.svg',dpi=350)
SSE = residual
diff = y - y.mean()
square_diff = diff ** 2
SST = square_diff.sum()

###  Now getting the coefficient of determination (R2)
R2 = 1 - SSE/SST 
print(R2)

Creating RawArray with float64 data, n_channels=1, n_times=1000000
    Range : 0 ... 999999 =      0.000 ...   500.000 secs
Ready.
[0.99263584]


In [18]:
plt.figure(figsize=(9.2,5))
plt.rc('font', size=16) #controls default text size
plt.plot(x,y)
plt.xlabel('lg(window size/sec)')
plt.ylabel('lg(flucturation)')
poly1d_fn = np.poly1d(model) 
plt.plot(x,y, 'ro', x, poly1d_fn(x))
plt.show()
plt.savefig('/mnt/user/drive/My Libraries/tutorials&explorations/V1criticality/data/2022-07_15_randm2_results.svg',dpi=350)